<center>
    <h1> Programación Científica</h1>
    <h2> Actividad 4 </h2>
    <h2> Felipe Chacón Ossa </h2>
    <h2> 201303017-3 </h2>
</center>

_ Abril 2017_

In [1]:
import numba
import numpy as np
import numexpr as ne
import matplotlib.pyplot as plt

En esta actividad implementaremos una conocida métrica para medir disimilitud entre conjuntos: __La métrica de Hausdorff__. Esta corresponde a un métrica o distancia ocupada para medir cuán disímiles son dos subconjuntos dados. 


Esta tiene muchas aplicaciones, en particular para comparar el parecido entre imágenes. En el caso en donde los conjuntos son arreglos bidimensionales, la definición es la siguiente:

Sean $X \in \mathbb{R}^{m \times 3}$ e  $Y \in \mathbb{R}^{n \times 3}$ dos matrices, la métrica/distancia de Hausdorff sobre sobre estas como:

$$
d_H(X,Y) = \max \left(\ \max_{i\leq m} \min_{j \leq n} d(X[i],Y[j]), \ \max_{j\leq n} \min_{i \leq m} d(Y[j],X[i]) \ \right)
$$

donde $d$ es la _distancia Euclideana_ clásica. ($X[i]$ indíca la i-ésima fila de X).

__Ilustración unidimensional:__ Distancia entre funciones.
<img src='data/hausdorff.png' style="width: 600px;">

1. Implemente la métrica de Hausdorff en Python clásico.
2. Implemente la métrica de Hausdorff usando Numba (Forzando el modo __nopython__ y definiendo explícitamente las _signatures_ de las funciones).
3. Cree `10` arreglos $X,Y$ aleatorios, con cantidad creciente de filas, y realice análsis de tiempos de ejecuciones de las funciones anteriores sobre estos arreglos.
4. Concluya.

In [26]:
def distance(a,b):
    return np.linalg.norm(a) - np.linalg.norm(b)

def hausdorff(a,b):
    l1, l2, l3 = [], [], []
    
    for i in range(a.shape[0]):
        for j in range(b.shape[0]):
            l1.append(distance(a[i],b[j]))
        l2.append(min(l1))
        l1 = []
    l3.append(max(l2))
    l2 = []
    
    for j in range(b.shape[0]):
        for i in range(a.shape[0]):
            l1.append(distance(b[j],a[j]))
        l2.append(min(l1))
        l1 = []
    l3.append(max(l2))
    l2 = []
    
    return(max(l3))

In [53]:
@numba.jit('float64 (float64[:], float64[:])', nopython = True)
def numbaDistance(a,b):
    return np.linalg.norm(a) - np.linalg.norm(b)

@numba.jit('float64 (float64[:,:], float64[:,:])')
def numbaHausdorff(a,b):
    l1, l2, l3 = [99999999], [-1], [-1]
    
    for i in range(a.shape[0]):
        for j in range(b.shape[0]):
            l1.append(numbaDistance(a[i],b[j]))
        l2.append(min(l1[1:]))
        l1 = [99999999]
    l3.append(max(l2))
    l2 = [-1]
    
    for j in range(b.shape[0]):
        for i in range(a.shape[0]):
            l1.append(numbaDistance(b[j],a[j]))
        l2.append(min(l1[1:]))
        l1 = [9999999]
    l3.append(max(l2))
    l2 = [-1]
    
    return(max(l3))

In [28]:
matricesX = []
matricesY = []
for i in np.arange(1,11):
    matricesX.append(np.random.random((15*i,15*i)))
    matricesY.append(np.random.random((15*i,15*i)))

In [45]:
%%timeit
for i in np.arange(10):
    hausdorff(matricesX[i],matricesY[i])

1 loop, best of 3: 1.38 s per loop


In [49]:
%%timeit
for i in np.arange(10):
    numbaHausdorff(matricesX[i],matricesY[i])

1 loop, best of 3: 264 ms per loop


Compilación Just In Time es excelente para acortar el tiempo de los algoritmos, pero no todos pueden ser utilizados con numba, además, ya que se traduce directamente a código de máquina, es mejor si se le explicitan los tipos de datos en vez de que los tenga que adivinar.